# IMPORT LIBRARIES

In [ ]:
import cv2
import tensorflow.keras
from PIL import Image, ImageOps
import numpy as np

import tkinter as tk
from tkinter import Message ,Text

import datetime
import time
import tkinter.ttk as ttk
import tkinter.font as font

# GETTING THE METER IMAGE

In [ ]:
image = cv2.imread("D:\FINAL YEAR PROJECTS\models\gray  model\FINAL\ENERGYMETER.jpeg")
cv2.imshow("meter reading", image) 
cv2.waitKey(0)
cv2.destroyAllWindows() 

# CANNY EDGE DETECTION

In [ ]:
image = cv2.imread("D:\FINAL YEAR PROJECTS\models\gray  model\FINAL\ENERGYMETER.jpeg") 
gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY) 
edged = cv2.Canny(image, 10, 250) 
cv2.imshow("edge", edged) 
cv2.waitKey(0)
cv2.destroyAllWindows() 
(cnts, _) = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2:]
idx = 0 
for c in cnts: 
	x,y,w,h = cv2.boundingRect(c) 
	if w>400 and h>90: 
		idx+=1 
		new_img=image[y:y+h,x:x+w] 
		cv2.imwrite( 'D:\FINAL YEAR PROJECTS\models\gray  model\FINAL\canny'+str(idx)+'.png', new_img) 
cv2.imshow("im",new_img) 
cv2.waitKey(0) 
cv2.destroyAllWindows() 

# CROPPING AND APPLYING MONOCHROME FILTER

In [51]:
image_file = Image.open("D:\FINAL YEAR PROJECTS\models\gray  model\FINAL\canny1.png") # open colour image
image_file = image_file.convert('L') # convert image to black and white
image_file = image_file.convert('L')
image_file.save('result.png')
img = cv2.imread('result.png')
x=0
img1 =img[60:152, 4:361]
cv2.imshow("show",img1)
print(img1.shape)
cv2.imwrite("D:\FINAL YEAR PROJECTS\models\gray  model\FINAL\monocropped.jpg",img1)
cv2.waitKey(0)
cv2.destroyAllWindows()

(92, 357, 3)


# IMAGE SEGMENTATION

In [52]:
im = cv2.imread("D:\FINAL YEAR PROJECTS\models\gray  model\FINAL\monocropped.jpg")
j = int((im.shape[1])/7)
h = int(im.shape[0])
z=0
for i in range(0,7):
    crop = im[0:92 , z:z+j]   
    crop =cv2.resize(crop,(224,224))
    cv2.imwrite('D:\FINAL YEAR PROJECTS\models\gray  model\FINAL\ '+str(i)+".jpg",crop)
    z+=j
    cv2.imshow("crop"+str(i),crop)
    cv2.waitKey(0)

cv2.destroyAllWindows() 

# PREDICTION MODEL 

In [53]:
np.set_printoptions(suppress=True)
model = tensorflow.keras.models.load_model('D:\FINAL YEAR PROJECTS\models\gray  model\FINAL\gray.h5')
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
im = cv2.imread("D:\FINAL YEAR PROJECTS\models\gray  model\FINAL\monocropped.jpg")
j = int(im.shape[1]/7)
z=0
unit =0
for i in range(0,7):
    crop = im[0:92 , z:z+j]    
    crop =cv2.resize(crop,(224,224))    
    cv2.imwrite(str(i)+".jpg",crop)    
    image = Image.open(str(i)+'.jpg')
    #resize the image to a 224x224 with the same strategy:
    #resizing the image to be at least 224x224 and then cropping from the center
    size = (224, 224)
    image = ImageOps.fit(image, size, Image.ANTIALIAS)
    
    #turn the image into a numpy array
    image_array = np.asarray(image)
    
    # display the resized image
    #image.show()
    
    # Normalize the image
    normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1
    
    # Load the image into the array
    data[0] = normalized_image_array
    
    # run the inference
    prediction = model.predict(data)
    #print(prediction)
    value = int(np.argmax(prediction,axis=1))
    unit+=(value*pow(10,6-i))
    print(value)
    z+=j
print(unit)

2
9
4
5
8
4
0
2945840


# SLAB CONTROL

In [54]:
premonthvalue = 2945540
noofdays = 30
unit = unit - premonthvalue
def unitcalc(unit1):    
    if(unit1<=100):
        bill = 0
        return bill
    elif(unit1<=200):
        bill = (unit1 -100)*1.50 +20
        #print(customerName+" Electricity Bill : Rs. "+str(bill) )
        #print(customerName+" had consumed "+str(per)+" % of slab 2")
        return bill
    elif(unit1<=500):
        unit1 -= 200
        bill = 230+(unit1*3)
        #print(customerName+" Electricity Bill : Rs. "+str(bill) )
        #print(customerName+" had consumed "+str(per)+" % of slab 3")
        return bill
    else:
        unit1 -=500
        bill = 1780+(unit1*6.6)
        #print(customerName+" Electricity Bill : Rs. "+str(bill))
        return bill
    
def perctangecalc(unit1):
    if(unit1<=100):
        per =  unit1
        return per,1
    elif(unit1<=200):
        per = int(unit1-100)
        return per,2
    elif(unit1<=500):
        unit1 -= 200
        per = int((unit1/300)*100)
        return per,3
    
def getunit(cost):
    if(cost<1130):
        newunit = (cost - 230)/3
        newunit +=200
        return newunit
    else:
        newunit = (cost - 1780)/6.6
        newunit +=500
        return newunit

consumed=unitcalc(unit)
percent,slab = perctangecalc(unit)
estimatedunit = (60/noofdays)*unit
estimatedcost = unitcalc(estimatedunit)
print("Electricity Bill:"+str("Rs. ")+str(consumed))
print("consumed "+str(percent)+str(" %")+" of energy at slab "+str(slab) )
print("Estimated Bill : "+str("Rs. ") + str(estimatedcost) )

Electricity Bill:Rs. 530
consumed 33 % of energy at slab 3
Estimated Bill : Rs. 2440.0


In [43]:
reducecost = 800   #BASE AMOUNT WITH 780 (765 CASE 2)
reduceunit = getunit(reducecost)
avalunit = reduceunit - unit
print(reduceunit)
print(avalunit)
remainingunit = estimatedunit-unit
print(consumed, percent, estimatedcost, estimatedunit, remainingunit )
##### for load forecast######
noofdays = 30
remdays = 60 - noofdays
dicthour = {
    "light" : 4,
    "fan" : 20,
    "charger" : 3,
    "motor" : 0.5,
    "tv" : 6,
    "refrigerator" : 24,
    "wetgrinder" : 0.5,
    "ironbox" : 0.5,
    "mixer" : 0.0166   
    }


dictload = {
    "light" : 0.040,
    "fan" : 0.075,
    "charger" : 0.007,
    "motor" : 0.750,
    "tv" : 0.100,
    "refrigerator" : 0.0356,
    "wetgrinder" : 0.300,
    "ironbox" : 0.700,
    "mixer" : 0.500   
    }  #all the load in kW

unitperday = 0
for x in dicthour:
    unitperday += dictload[x]* dicthour[x]
print(unitperday*remdays)
basicunit = unitperday*remdays
additionalunit = (avalunit - basicunit)/remdays
print("Additonal unit "+ str(additionalunit))
print(unitperday)
invalidnumber = 0
flag=0
if(additionalunit>0):
    flag = 1
    amt = unitcalc(basicunit+unit)
    amt = reducecost-amt
    print("Amount saved :"+str(amt))   
if(additionalunit<0):
    additionalunit = unitperday+additionalunit
    if(additionalunit>1.67803): 
        runit = additionalunit - 1.67803
        if(runit>0.9):
                #dicthour["wetgrinder"]= 0.1428
                #dicthour["ironbox"]= 0.1428
                #dicthour["mixer"]= 0.0833
                dicthour["fan"] = 8
                dicthour["tv"] = 3
                temp = dicthour["fan"]*dictload["fan"] + dicthour["tv"]*dictload["tv"]
                runit -=temp
                if(runit>0.1428):
                    dicthour["wetgrinder"]= 0.1428
                    dicthour["ironbox"]= 0.1428
                    runit -=0.1428
                    additionaltv = runit/dictload["tv"]
                    additionalfan = runit/dictload["fan"]
                    print(additionaltv,additionalfan)
                else:
                    dicthour["wetgrinder"] = 0
                    dicthour["ironbox"] = 0
                    additionaltv = runit/dictload["tv"]
                    additionalfan = runit/dictload["fan"]
                    print(additionaltv,additionalfan)
        else:
            invalidnumber = 1
            print("Invalid amount")
    else:
        invalidnumber = 1
        print("Invalid amount")
if(not invalidnumber):
    for x,y in dicthour.items():
        print(str(x)+ "  :  "+str(y) )
else:
    print("Invalid amount")

390.0
90.0
530 33 2440.0 600.0 300.0
120.56099999999999
Additonal unit -1.0186999999999997
4.0187
2.7917 3.722266666666667
light  :  4
fan  :  8
charger  :  3
motor  :  0.5
tv  :  3
refrigerator  :  24
wetgrinder  :  0.1428
ironbox  :  0.1428
mixer  :  0.0166
